Importing the Data from **GitHub**

In [ ]:
!git clone https://github.com/muxspace/facial_expressions.git

In [ ]:
import csv
d={}
file = open('/content/facial_expressions/data/legend.csv')
filedata = csv.reader(file)

In [ ]:
for row in filedata:
  key = row[2].lower()
  value = row[1]
  if key in d:
    d[key].append(value)
  else:
    d[key]=[]
    d[key].append(value)


In [ ]:
#Deleting the column headers.
del d['emotion']

In [ ]:
emotion_list = list(d.keys())
emotion_list

Preparing the data and making the **Directories**

In [ ]:

#Making the masterdata folder which contains training&testing sub directories.
import os

os.mkdir('master_data')
os.mkdir('master_data/training_data')
os.mkdir('master_data/testing_data')

In [ ]:
#Copying the all 8 categories into training&testing folders of master_data.
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training_data/', emotion))
  os.mkdir(os.path.join('master_data/testing_data/', emotion))

In [ ]:
#Now copying the images from images_directory into the master_data training&testing folders
from shutil import copyfile
split_size=0.8
for emotions, images in d.items():
  train_size = int(split_size*len(images))
  train_images = images[0:train_size]
  test_images = images[train_size:]

  #Now copying these images.
  for each_image in train_images:
    source = os.path.join('/content/facial_expressions/images', each_image)
    dest = os.path.join('/content/master_data/training_data', emotion, each_image)
    copyfile(source, dest)

  for each_image in test_images:
    source = os.path.join('/content/facial_expressions/images', each_image)
    dest = os.path.join('/content/master_data/testing_data', emotion, each_image)
    copyfile(source, dest)

Now Using the ImageDataGenerator from Keras.


In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(100,100,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(8, activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Finally Providing the Images to our Model

In [ ]:
train_dir = '/content/master_data/training_data'
testing_dir = '/content/master_data/testing_data'

train_datagen = ImageDataGenerator(rescale=1.0/255)
train_gen = train_datagen.flow_from_directory(train_dir, target_size=(100,100), class_mode='categorical', batch_size=64)

test_datagen = ImageDataGenerator(rescale=1.0/255)
test_gen = test_datagen.flow_from_directory(testing_dir, target_size=(100,100), class_mode='categorical', batch_size=64)

In [ ]:
es = EarlyStopping(monitor='val_acc', patience=5, min_delta=0.05)

In [ ]:
model.fit_generator(train_gen, epochs=10, validation_data=test_gen, verbose=1, callbacks=[es])